In [1]:
#!/usr/bin/env python

import trappy
from trace import Trace
import itertools
import logging
import pandas as pd
import numpy as np
import os

from conf import LisaLogging
LisaLogging.setup()
logging.info('#### Setup FTrace')
# path_to_html = "/home/joelaf/repo/lisa-aosp/external/lisa/results/UiBench_default/trace.html"
path_to_html = "/home/joelaf/repo/lisa-aosp/external/lisa/ipynb/residency/trace-residency.html"

2017-07-10 12:26:40,568 INFO    : root         : Using LISA logging configuration:
2017-07-10 12:26:40,570 INFO    : root         :   /home/joelaf/repo/lisa-aosp/external/lisa/logging.conf
2017-07-10 12:26:40,571 INFO    : root         : #### Setup FTrace


In [2]:
tr = Trace(None, path_to_html,
           cgroup_info = { 
               'cgroups': ['foreground', 'background', 'system-background', 'top-app', 'rt'],
               'controller_ids': { 2: 'cpuset', 4: 'schedtune' }
           },
           events=[ 'sched_switch', 'cgroup_attach_task_devlib', 'cgroup_attach_task', 'sched_process_fork' ],
        normalize_time=False)

# tr.data_frame.trace_event('cgroup_attach_task_devlib')
# len(tr.data_frame.trace_event('cgroup_attach_task_devlib').groupby(level=0).filter(lambda x: len(x) > 1))
cdf = tr.data_frame.cgroup_attach_task()

ssdf = tr.data_frame.sched_switch_cgroup()

/home/joelaf/repo/lisa-aosp/external/lisa/libs/utils/trace.py:258: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  window=window, normalize_time=normalize_time)
/home/joelaf/repo/lisa-aosp/external/trappy/trappy/utils.py:154: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  df = pd.concat([pr_df, sec_df], keys=['primary', 'secondary']).sort(columns='__line')
/home/joelaf/repo/lisa-aosp/external/lisa/libs/utils/trace.py:574: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  fdf = pd.concat([fdf, forkdf]).sort(columns='__line')
/home/joelaf/repo/lisa-aosp/external/lisa/libs/utils/trace.py:584: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  fdf = pd.concat([fdf, cdf]).sort(columns='__line')
2017-07-10 12:26:44,985 INFO    : Trace        : Couldn't attach all forks cgroup with-attach events (57 dropped)
2017-07-10 12:26:45,097 INFO    : Trace        : 

In [3]:
# build schedtune group information

# cdf[cdf['cgroup'] == 'top-app']
# ssdf = sdf[sdf['next_pid'] == 18591]
# ssdf = ssdf.loc[ssdf.index > 18003.5]

df = ssdf
df = df.loc[df.index > 317.5]
df = df.loc[(df['next_pid'] != 0) & (df['prev_pid'] != 0)]
# df = df.loc[ (pd.isnull(df['prev_schedtune'])) | (pd.isnull(df['prev_cpuset'])) | \
#              (pd.isnull(df['next_schedtune'])) | (pd.isnull(df['next_cpuset'])) ]

df = df[['prev_pid', 'prev_comm', 'prev_cpuset', 'prev_schedtune']].loc[pd.isnull(df['prev_cpuset'])]


forkdf = tr.data_frame.trace_event('sched_process_fork')
forkdf = forkdf.loc[forkdf.index > 317.5]
 
cdf = tr.data_frame.cgroup_attach_task()
cdf = cdf.loc[cdf.index > 317.5]

print ''
print 'original fork df:'
print forkdf.head(20)[['pid', 'child_pid']]
print ''

print 'combined cgroup df'
print cdf.head(20)
print ''

print 'final df with nans'
print df

# ssdf = ssdf.loc[(ssdf['prev_pid'] == 18591) | (ssdf['next_pid'] == 18591)]
# print ssdf[['prev_pid', 'prev_comm', 'prev_schedtune', 'prev_cpuset']]
# print ssdf[['next_pid', 'next_comm', 'next_schedtune', 'next_cpuset']]

2017-07-10 12:27:04,224 INFO    : Trace        : Couldn't attach all forks cgroup with-attach events (57 dropped)



original fork df:
             pid  child_pid
Time                       
317.512955  2467       9883
324.385811  1086       9884
325.493887  9696       9885
325.494522  9695       9886
325.495726  9696       9887
325.496083  9696       9888
325.496302  9696       9889
325.516582  9885       9890
325.518904  1152       9891
326.379281  8569       9892
326.822364  1042       9893
331.001721     2       9894
336.826480  1042       9895
337.580295  1166       9896
337.587107  1166       9897
337.605980  9896       9898
337.650938  1166       9899
337.657055  1166       9900
337.670824  9899       9901
337.672171  9901       9902

combined cgroup df
            __line     pid controller      cgroup
Time                                             
317.512955   54833  9883.0  schedtune  foreground
317.512955   54833  9883.0     cpuset        root
321.554727  136229  1481.0     cpuset  foreground
321.581165  136824  1481.0     cpuset  background
321.581535  136828  1481.0     cpuset  backgr

In [7]:
# df = ssdf.loc[ssdf.index > 317.5]

df = tr.data_frame.sched_switch_cgroup()
df.loc[df.index > 317.5]

2017-07-10 12:36:40,457 INFO    : Trace        : Couldn't attach all forks cgroup with-attach events (57 dropped)


,__comm,__cpu,__line,__pid,__tgid,prev_cpuset,next_comm,next_cpuset,next_pid,next_prio,next_schedtune,prev_pid,prev_comm,prev_prio,prev_schedtune,prev_state
Time,,,,,,,,,,,,,,,,
317.500003,lowpool[12],6.0,53823,7897.0,2467.0,root,Binder:1141_1,foreground,1152.0,120.0,foreground,7897.0,lowpool[12],130.0,foreground,R
317.500011,.gms.persisten,7.0,53824,2467.0,2467.0,root,Binder:1141_3,foreground,1953.0,120.0,foreground,2467.0,.gms.persistent,120.0,foreground,S
317.500040,lowpool[9],2.0,53826,5668.0,2467.0,root,Binder:1141_2,foreground,1153.0,120.0,foreground,5668.0,lowpool[9],130.0,foreground,S
317.500088,Binder:1141_1,6.0,53828,1152.0,1141.0,foreground,lowpool[12],root,7897.0,130.0,foreground,1152.0,Binder:1141_1,120.0,foreground,S
317.500099,Binder:1141_3,7.0,53829,1953.0,1141.0,foreground,FlpThread,root,3032.0,120.0,foreground,1953.0,Binder:1141_3,120.0,foreground,S
317.500112,Binder:1141_7,0.0,53831,2568.0,1141.0,foreground,lmkd,root,1021.0,98.0,system-background,2568.0,Binder:1141_7,118.0,foreground,R+
317.500117,lowpool[16],1.0,53834,9475.0,2467.0,root,swapper/1,NaN,0.0,120.0,NaN,9475.0,lowpool[16],130.0,foreground,S
317.500194,FlpThread,7.0,53837,3032.0,2467.0,root,Binder:1141_3,foreground,1953.0,120.0,foreground,3032.0,FlpThread,120.0,foreground,S
317.500206,lmkd,0.0,53838,1021.0,1021.0,root,Binder:1141_7,foreground,2568.0,118.0,foreground,1021.0,lmkd,98.0,system-background,S
